In [1]:
import numpy as np
import scipy as sp
import scipy.io
from scipy.stats import unitary_group
from datetime import datetime
import matplotlib.pyplot as plt
import qutip as qt
#from scipy.optimize import curve_fit
import os
import uuid
from itertools import repeat, chain, product, combinations
from joblib import Parallel, delayed
import glob
import time
from EMQST_lib import support_functions as sf
from EMQST_lib import emqst
from EMQST_lib.qst import QST  
from EMQST_lib.povm import POVM
from EMQST_lib import dt
from EMQST_lib import measurement_functions as mf
from EMQST_lib import visualization as vis
from EMQST_lib import overlapping_tomography as ot
from EMQST_lib import photon as ph
from functools import reduce
from cvxopt import matrix, solvers
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=2)

Comparing integrated coincidence counts to the background counts, counts on each beam line before before integration starts

In [33]:
is_math_file = True
if is_math_file:
    
    path  = "Photon_REM/photonic QREM experimental data v3/13 HVVH with lens/HVVH_lens.mat"
    compare_path_lens = "Photon_REM/photonic QREM experimental data v3-singles/13 HVVH with lens/HVVH_lens_singles.mat"
    compare_path_no_lens = "Photon_REM/photonic QREM experimental data v3-singles/14 HVVH no lens final check/HVVH_singles.mat"
    mat_files = glob.glob(path)[::-1] # To get order HVVH, HLVR, HDVA
    compare_files = glob.glob(compare_path_lens)[::-1] # To get order HVVH, HLVR, HDVA
    compare_files_no_lens = glob.glob(compare_path_no_lens)[::-1] # To get order HVVH, HLVR, HDVA
    print(mat_files)
    print(compare_files)
    print(compare_files_no_lens)
    #mat_files = [glob.glob(p + "/*.mat")[0] for p in paths]

    # Load from math files
    QST_dic = [sp.io.loadmat(f) for f in mat_files]
    #print(QST_dic[0]['HVVH_lens_coincRaw'])
    #print(QST_coinc_count_dic)
    QST_coinc_keys = [list(mat.keys()) for mat in QST_dic]
    print(QST_coinc_keys)
    # QST_calib_keys = [list(mat.keys())[3] for mat in QST_dic]
    # #print(QST_coinc_keys)
    # QST_coinc_counts = np.array([mat[key].T[0] for mat, key in zip(QST_dic, QST_coinc_keys)])
    
    compare_dic = [sp.io.loadmat(f) for f in compare_files]
    compare_path_no_lens = [sp.io.loadmat(f) for f in compare_files_no_lens]
    #print(compare_dic[0]["HVVH_lens_singles"])
    # compare_QST_coinc_keys = [list(mat.keys())[6] for mat in compare_dic]
    # compare_QST_calib_keys = [list(mat.keys())[3] for mat in compare_dic]
    # compare_QST_coinc_counts = np.array([mat[key].T[0] for mat, key in zip(compare_dic, compare_QST_coinc_keys)])

coincidene_order_QST = ['HH', 'HV', 'VV', 'VH',
                        'RH', 'RV', 'DV', 'DH',
                        'DR', 'DD', 'RD', 'HD',
                        'VD', 'VL', 'HL', 'RL']

for i in range(len(coincidene_order_QST)):
    print(coincidene_order_QST[i], QST_dic[0]['HVVH_lens_coincRaw'][i] ,QST_dic[0]['HVVH_lens_coincAcc'][i] , compare_dic[0]["HVVH_lens_singles"][i], compare_path_no_lens[0]["HVVH_singles"][i])



['Photon_REM/photonic QREM experimental data v3/13 HVVH with lens/HVVH_lens.mat']
['Photon_REM/photonic QREM experimental data v3-singles/13 HVVH with lens/HVVH_lens_singles.mat']
['Photon_REM/photonic QREM experimental data v3-singles/14 HVVH no lens final check/HVVH_singles.mat']
[['__header__', '__version__', '__globals__', 'HVVH_lens', 'HVVH_lens_coinc', 'HVVH_lens_coincAcc', 'HVVH_lens_coincRaw']]
HH [1208] [734.05] [11826  9076] [11997 23234]
HV [44567] [1178.65] [11971  8928] [11892 20999]
VV [1029] [594.05] [9583 8713] [ 9352 21171]
VH [15002] [705.68] [9356 8789] [ 9442 22993]
RH [8811] [743.51] [10446  8804] [10662 22810]
RV [23029] [883.51] [10546  8910] [10673 20990]
DV [23426] [893.92] [10597  8782] [10527 20616]
DH [8489] [743.51] [10520  8758] [10531 23034]
DR [24248] [891.08] [10641  8806] [10446 21228]
DD [25508] [947.84] [10530  8560] [10593 21789]
RD [8598] [753.92] [10688  8673] [10843 21704]
HD [22071] [920.41] [11750  8826] [11804 21711]
VD [9303] [665.] [9503 879